In [1]:
# coding: utf-8
# 用gensim去做word2vec的处理，用sklearn当中的SVM进行建模
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd
import jieba
from sklearn.externals import joblib
from sklearn.svm import SVC
from importlib import reload
import sys
reload(sys)
import _pickle as cPickle
import jieba.analyse as ana

C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
sys.getdefaultencoding()

'utf-8'

In [3]:
stopwords=pd.read_table('stopwords.txt',names=['words'],encoding='utf-8')
stopwords.head() 
#  载入数据，做预处理(分词)，切分训练集与测试集
def load_file_and_preprocessing():
    neg=pd.read_excel('data/neg.xls',header=None,index=None)
    pos=pd.read_excel('data/pos.xls',header=None,index=None)
    postive=pd.read_excel('data/pos.xls',header=None,index=None)
    negtive=pd.read_excel('data/neg.xls',header=None,index=None)
    cw = lambda x: list(jieba.cut(x))

    # 新增一列 word ,存放分好词的评论，pos[0]代表表格第一列

    pos['words'] = pos[0].apply(cw)
    neg['words'] = neg[0].apply(cw)
    postive['words'] = pos[0].apply(cw)
    negtive['words'] = neg[0].apply(cw)
    print(pos['words'])
    print(neg['words'])
    

    for i in range(len(pos['words'])):
        postive['words'][i]= [word for word in pos['words'][i] if word not in list(stopwords['words'])]
    for i in range(len(neg['words'])):
        negtive['words'][i]= [word for word in neg['words'][i] if word not in list(stopwords['words'])]
    print(postive['words'])
    print(negtive['words'])
#     ana.set_stop_words('stopwords.txt')
#     sentence='大数据专业的同学棒棒哒！'
#     print(ana.extract_tags(pos['words'][0]))
    # np.ones(len(pos)) 新建一个长度为len(pos)的数组并初始化元素全为1来标注好评
    # np.concatenate（）连接数组
    # axis=0 向下执行方法 axis=1向右执行方法
    y = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))),axis=0)

    # train_test_split：从样本中随机的按比例选取train data和testdata
    # 一般形式：train_test_split(train_data,train_target,test_size=0.4, random_state=0)
    # train_data：所要划分的样本特征集
    # train_target：所要划分的样本结果（标注）
    # test_size：样本占比，如果是整数的话就是样本的数量
    # random_state：是随机数的种子。

    x_train, x_test, y_train, y_test = train_test_split(np.concatenate((postive['words'], negtive['words'])), y, test_size=0.2)
    np.save('svm_data/y_train.npy',y_train)
    np.save('svm_data/y_test.npy',y_test)
    a=np.load('svm_data/y_train.npy')
    print(a)
    return x_train,x_test



# 对每个句子的所有词向量取均值，来生成一个句子的vector
def build_sentence_vector(text, size,imdb_w2v):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

# 计算词向量
def get_train_vecs(x_train, x_test):
    n_dim = 300
    # 初始化模型和词表
    imdb_w2v = Word2Vec(x_train, size=n_dim, min_count=10)
    # imdb_w2v = Word2Vec(size=300, window=5, min_count=10, workers=12)
    # imdb_w2v.build_vocab(x_train)
    #
    # imdb_w2v.train(x_train,
    #                total_examples=imdb_w2v.corpus_count,
    #                epochs=imdb_w2v.iter)


    train_vecs = np.concatenate([build_sentence_vector(z, n_dim, imdb_w2v) for z in x_train])
    # train_vecs = scale(train_vecs)

    np.save('svm_data/train_vecs.npy', train_vecs)
    print(train_vecs.shape)
    # 在测试集上训练
    #imdb_w2v.train(x_test)
    imdb_w2v.train(x_test,
                   total_examples=imdb_w2v.corpus_count,
                   epochs=imdb_w2v.iter)

    imdb_w2v.save('svm_data/w2v_model/w2v_model.pkl')
    # Build test tweet vectors then scale
    test_vecs = np.concatenate([build_sentence_vector(z, n_dim, imdb_w2v) for z in x_test])
    # test_vecs = scale(test_vecs)
    np.save('svm_data/test_vecs.npy', test_vecs)
    print(test_vecs.shape)



def get_data():
    train_vecs=np.load('svm_data/train_vecs.npy')
    y_train=np.load('svm_data/y_train.npy')
    test_vecs=np.load('svm_data/test_vecs.npy')
    y_test=np.load('svm_data/y_test.npy')
    return train_vecs,y_train,test_vecs,y_test

# 训练svm模型

def svm_train(train_vecs,y_train,test_vecs,y_test):
    clf=SVC(kernel='rbf',verbose=True)
    clf.fit(train_vecs,y_train)
    joblib.dump(clf, 'svm_data/svm_model/model.pkl')
    print(clf.score(test_vecs,y_test))


# 构建待预测句子的向量

def get_predict_vecs(words):
    n_dim = 300
    imdb_w2v = Word2Vec.load('svm_data/w2v_model/w2v_model.pkl')
    #imdb_w2v.train(words)
    train_vecs = build_sentence_vector(words, n_dim,imdb_w2v)
    #print(train_vecs.shape)
    return train_vecs

# 对单个句子进行情感判断
def svm_predict(string):
    words=jieba.lcut(string)
    words = [word for word in words if word not in list(stopwords['words'])]
    words_vecs=get_predict_vecs(words)
    clf=joblib.load('svm_data/svm_model/model.pkl')
    print(clf)
     
    result=clf.predict(words_vecs)
    

    if int(result[0])==1:
        print(string,' positive')
        global poscnt
        poscnt += 1

    if int(result[0])==0:
        print(string,' negtive')
        global negcnt
        negcnt = negcnt + 1


#
x_train,x_test = load_file_and_preprocessing()
get_train_vecs(x_train,x_test)
train_vecs,y_train,test_vecs,y_test = get_data()
svm_train(train_vecs,y_train,test_vecs,y_test)

#对输入句子情感进行判断
string='涨'
# string='沪指震荡跌0.36% 疫苗概念大幅走低'
data=pd.read_excel('data/data.xls',header=None,index=None)
print(data)
poscnt=0
negcnt=0
for i in range(0, len(data)):
    svm_predict(str(data.iloc[i]))
print("total pos=%d" % poscnt)
print("total neg=%d" % negcnt)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\hasyo\AppData\Local\Temp\jieba.cache
Loading model cost 1.927 seconds.
Prefix dict has been built successfully.


0      [2, 万人, 围观, 中信证券, 策略, 会, ：, A股, 中期, 上行, 将, 在, ...
1      [8, 月, 31, 日, ，, 沪, 指跌, 0.24%, ，, 沪, 深, 300, 指...
2      [9, 月, 11, 日, ，, A股, 三大, 指数, 低开, 高, 走, ，, 创业板,...
3      [9, 月, 11, 日, ，, 两市, 股指, 早盘, 冲高, 回落, ，, 午后, 强劲...
4      [9, 月, 11, 日, 异动股, 点评, ：, 风电, 板块, “, 乘风破浪, ”, ...
                             ...                        
285                                                  [涨]
286                                                  [涨]
287                                                  [涨]
288                                                  [涨]
289                                                  [涨]
Name: words, Length: 290, dtype: object
0                                                   [跌]
1                                                   [跌]
2                                                   [跌]
3                                                   [跌]
4                                                   [跌]
5                                                   [

C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



(70, 300)
[LibSVM]0.9714285714285714
     0
0    涨
1    涨
2    涨
3    涨
4    涨
..  ..
205  涨
206  涨
207  涨
208  涨
209  涨

[210 rows x 1 columns]
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 0, dtype: object  positive
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 1, dtype: object  positive
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 2, dtype: obje

C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use se

...

C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use se

 positive
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 198, dtype: object  positive
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 199, dtype: object  positive
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 200, dtype: object  positive
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=

C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use se

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 208, dtype: object  positive
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
0    涨
Name: 209, dtype: object  positive
total pos=210
total neg=0


C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\hasyo\Anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
